In [1]:
import pandas as pd
from IPython.display import HTML
def printmd(string):
    from IPython.display import Markdown, display
    display(Markdown(string))

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

### postgresql + python ###
import psycopg2 as pg2

class psql_py:
    def __init__(self):
        self.conn = self.connect_db()
        self.cur = self.conn.cursor()
        self.cur.execute("set search_path = mimiciii;") 
        
    def connect_db(self):
        try:
            conn = pg2.connect("host = localhost dbname=mimic user=postgres port=5432")
            #print('database connected')
            return conn
        except:
            print("I am unable to connect to the database")          
            
    def exe_sql (self, exe_sql_str):
        return pd.read_sql(exe_sql_str,self.conn)
    
    def db_query(self, exe_sql_str, mode='sql'):
        if mode == 'pd':
            return_pd = self.exe_sql (exe_sql_str)
            self.close_db()
            return return_pd
        else: 
            self.cur.execute(exe_sql_str)
            self.close_db()
            
    def close_db (self):
        self.cur.close()
        # commit the changes
        self.conn.commit()
        #print('database disconnected')


In [36]:
dict_variable_id = pd.read_csv('./csv/item_ids_dict.csv') # variables related to extubation 

#supervised by Young-Jae Cho at Seoul National University Bundang Hostpital 

In [37]:
db = psql_py() 
print('extration each variable from chartevent')
for i, row in dict_variable_id.iterrows(): #from chartevent
    var_name = row['name']
    var_name = var_name.replace('/','_')
    var_name = var_name.replace(' ','_')
    var_name = var_name.replace('-','_')
    var_id = row['id']
    str_sql = ('select chartevents.icustay_id, chartevents.itemid, chartevents.charttime, chartevents.value '+
               'from chartevents, ptsinfo_vent '+ 
               'where chartevents.icustay_id = ptsinfo_vent.icustay_id and itemid in ('+str(var_id)+
              ') and ptsinfo_vent.admittime <= chartevents.charttime and ptsinfo_vent.dischtime >= chartevents.charttime')
    var_record = db.db_query(str_sql,'pd')
    var_record.to_csv('../Variables/chartevent_var_'+var_name+'.csv')
    print(var_name,var_id)
    
print('\n extration each variable from labevent')    
for i, row in dict_variable_id.iterrows(): #from labevent
    var_name = row['name']
    var_name = var_name.replace('/','_')
    var_name = var_name.replace(' ','_')
    var_name = var_name.replace('-','_')
    var_id = row['id']
    str_sql = ('select labevents.subject_id, labevents.itemid,labevents.charttime, labevents.value from labevents, ptsinfo_vent\
                 where labevents.subject_id = ptsinfo_vent.subject_id and itemid in ('+str(var_id)+
              ')and ptsinfo_vent.admittime <= labevents.charttime and ptsinfo_vent.dischtime >= labevents.charttime')
    var_record = db.db_query(str_sql,'pd')
    if len(var_record)>0:
        var_record.to_csv('../Variables/labevent_var_'+var_name+'.csv')
    print(var_name,var_id)
    
    

extration each variable from chartevent
GCS_total 198, 226755, 227013
GCS_eye_opening 220739, 226756, 227011, 184
GCS___Verbal_Response 223900, 228112, 226758, 227014, 723
GCS___Motor_Response 223901, 226757, 227012, 454
RASS 228096
Goal_Richmond_RAS_Scale 228299
CAM_ICU_MS_change 228300, 228337
CAM_ICU_Inattention 228301, 228336
CAM_ICU_RASS_LOC 228302
CAM_ICU_Disorganized_thinking 228303, 228335
Delirium_assessment 228332
CAM_ICU_Altered_LOC 228334, 228302
HR 211, 220045
MeanBP 220181, 220052, 225312, 224322, 6702, 443, 52, 456
RR 220210, 3337, 224422, 618, 3603, 615, 224690
SpO2_SaO2 220277, 646, 834
Temp_F 223761, 678, 679, 3652, 227054
CVP 220074, 113, 1103
Potassium_mEqL 829, 1535, 227442, 227464, 4194, 3725, 3792, 226535, 220640, 3725, 50971, 50822
Sodium 837, 220645, 3803, 226534, 1536, 4195, 3726, 50983, 50824
Chloride 788, 220602, 1523, 4193, 3724, 226536, 3747, 3724, 50902, 50806
Glucose 225664, 807, 811, 1529, 220621, 226537, 3744, 50809, 50931
BUN 781, 1162, 225624, 3737, 